In [1]:
import numpy as np
import latlng
import treefinder
from matplotlib import pyplot as plt
import roadnetwork

class Tree:
    def __init__(self, category, lat, lng):
        self.category = category
        self.lat = lat
        self.lng = lng


In [2]:
trees = []
filename = 'trees/2015StreetTreesCensus_TREES.csv'
f = open(filename, 'r')
t = 0
for line in f:
    if t == 0:
        t += 1
        continue
    temp = line.split(',')
    if len(temp) < 30:
        continue
    if temp[30] == "Manhattan":
        lat = float(temp[38])
        lng = float(temp[39])
        sp = temp[9]
        trees.append(Tree(sp,lat,lng))

trees.sort(key=lambda x:x.lat*1000+x.lng)


In [3]:
f = open('trees_man_all.csv','w')
for tree in trees:
    f.write(tree.category+';'+str(tree.lat)+';'+str(tree.lng)+'\n')
f.close()

In [4]:
rn = roadnetwork.RoadNetwork("road_network_man.xml")
tf = treefinder.TreeFinder("trees_man_all.csv")
print(len(tf.trees))
for road in rn.roads:
    cur_pos = road.get_pos_of(0)
    while True:
        ll = road.get_latlng(cur_pos)
        trees = tf.find_trees(ll.lat, ll.lng, 30)
        for tree in trees:
            tree.id = -1
        cur_pos = road.get_pos_from_to(cur_pos, 10)
        if cur_pos.tdis < 0:
            break
del_trees = []
tid = 1
for tree in tf.trees:
    if tree.id < 0:
        tree.id = tid
        tid += 1
    else:
        del_trees.append(tree)
print(len(tf.trees))
for tree in del_trees:
    tf.trees.remove(tree)
print(len(tf.trees))

57800
57800
57499


In [5]:
f = open('trees_man.csv','w')
for tree in tf.trees:
    f.write(tree.category+';'+str(tree.lat)+';'+str(tree.lng)+'\n')
f.close()